In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/DeepFundingMini

/content/drive/MyDrive/DeepFundingMini


In [ ]:
!wget -O deepfundutils.py https://raw.githubusercontent.com/hara-desu/DeepFundingMiniContest/refs/heads/main/code/deepfundutils.py

--2025-01-15 17:29:39--  https://raw.githubusercontent.com/hara-desu/DeepFundingMiniContest/refs/heads/main/code/deepfundutils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7197 (7.0K) [text/plain]
Saving to: ‘deepfundutils.py’

deepfundutils.py    100%[===================>]   7.03K  --.-KB/s    in 0s      

2025-01-15 17:29:39 (13.8 MB/s) - ‘deepfundutils.py’ saved [7197/7197]



In [3]:
import warnings
import pandas as pd
import numpy as np

from sklearn.ensemble import HistGradientBoostingRegressor
from deepfundutils import model_eval, fix_pred_range, label_encode_ab, make_triplets

# warnings.filterwarnings('ignore')

In [4]:
splits = {'train': 'train.parquet', 'test': 'test.parquet'}

df_train = pd.read_parquet('hf://datasets/evalscience/deepfunding-mini/' + splits['train'])
df_valid = pd.read_parquet('hf://datasets/evalscience/deepfunding-mini/' + splits['test'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Query OSO data

In [5]:
projects_ab_train = tuple(
    pd.concat(
        [df_train['project_a'], df_train['project_b']]
    ).unique()
)

projects_ab_valid = tuple(
    pd.concat(
        [df_valid['project_a'], df_valid['project_b']]
    ).unique()
)

In [6]:
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()

In [7]:
# Querying data for train dataset

# replace with your project name ↘↘↘
%%bigquery df_oso_train --project hip-transducer-446111-g7

with watcher_count as (
  select
  project_id,
  artifact_id,
  artifact_url,
  watcher_count
from `oso_production.repositories_v0`
),

artifact_webpage as (
  select
    project_id,
    min(artifact_name) as webpage
  from `oso_production.artifacts_by_project_v1`
  where (artifact_source = 'WWW')
    and (artifact_name not like '%https://opencollective.com/%')
    and (artifact_name not like '%https://github.com/%')
    and (artifact_name not like '%http://discord.gg%')
  group by project_id
),

artifact_twitter as (
  select
    project_id,
    min(artifact_name) as twitter
  from `oso_production.artifacts_by_project_v1`
  where artifact_source = 'TWITTER'
  group by project_id
),

funding as (
  select
    count(distinct grant_pool_name) as count_all_grant_pools,
    to_project_id
  from `oso_production.oss_funding_v0`
  group by to_project_id
),

github_metrics as (
  select
    project_id,
    repository_count,
    developer_count,
    contributor_count
  from `oso_production.code_metrics_by_project_v1`
)

select
  watcher_count.artifact_url as url,
  watcher_count.watcher_count,
  artifact_webpage.webpage,
  artifact_twitter.twitter,
  funding.count_all_grant_pools,
  github_metrics.repository_count,
  github_metrics.developer_count,
  github_metrics.contributor_count
 from watcher_count
    left join artifact_webpage
      on watcher_count.project_id = artifact_webpage.project_id
    left join artifact_twitter
      on watcher_count.project_id = artifact_twitter.project_id
    left join funding
      on watcher_count.project_id = funding.to_project_id
    left join github_metrics
      on watcher_count.project_id = github_metrics.project_id

  -- Add the list of relevant repo_urls as a where clause
    where artifact_url in ('https://github.com/prettier-solidity/prettier-plugin-solidity','https://github.com/prysmaticlabs/prysm','https://github.com/sigp/lighthouse','https://github.com/walletconnect/walletconnect-monorepo','https://github.com/nomicfoundation/hardhat','https://github.com/vyperlang/vyper','https://github.com/wighawag/hardhat-deploy','https://github.com/ethers-io/ethers.js','https://github.com/ethereum/solidity','https://github.com/ethereum/remix-project','https://github.com/ethereum/go-ethereum','https://github.com/consensys/teku','https://github.com/wevm/viem','https://github.com/chainsafe/lodestar','https://github.com/openzeppelin/openzeppelin-contracts','https://github.com/protofire/solhint','https://github.com/web3/web3.js','https://github.com/ipfs/js-ipfs','https://github.com/mochajs/mocha','https://github.com/gulpjs/gulp','https://github.com/webpack/webpack','https://github.com/redux-saga/redux-saga','https://github.com/debug-js/debug','https://github.com/chzyer/readline','https://github.com/vuejs/vue','https://github.com/marak/colors.js','https://github.com/reactivex/rxjs','https://github.com/webreflection/flatted','https://github.com/xtuc/webassemblyjs','https://github.com/rollup/rollup','https://github.com/level/levelup','https://github.com/mikemcl/bignumber.js','https://github.com/go-task/slim-sprig','https://github.com/electron/electron','https://github.com/gregberge/svgr','https://github.com/emotion-js/emotion','https://github.com/pnpm/cmd-shim','https://github.com/wooorm/markdown-table','https://github.com/coinbase/coinbase-wallet-sdk','https://github.com/swc-project/swc','https://github.com/bradfitz/iter','https://github.com/sindresorhus/type-fest','https://github.com/eslint/eslint','https://github.com/zloirock/core-js','https://github.com/sheetjs/js-crc32','https://github.com/dcodeio/long.js','https://github.com/qix-/color-convert','https://github.com/mafintosh/pump','https://github.com/prettier/prettier','https://github.com/typescript-eslint/typescript-eslint','https://github.com/salesforce/tough-cookie','https://github.com/pytest-dev/pytest','https://github.com/heim-rs/darwin-libproc','https://github.com/immerjs/immer','https://github.com/jedisct1/go-minisign','https://github.com/lukeed/polka','https://github.com/facebook/react','https://github.com/cssnano/cssnano','https://github.com/motdotla/dotenv','https://github.com/pion/webrtc','https://github.com/postcss/postcss','https://github.com/babel/babel','https://github.com/mozilla/source-map','https://github.com/floating-ui/floating-ui','https://github.com/browserslist/browserslist','https://github.com/node-fetch/node-fetch','https://github.com/mattn/go-isatty','https://github.com/clap-rs/clap','https://github.com/yarnpkg/yarn','https://github.com/fb55/entities','https://github.com/ljharb/qs','https://github.com/epoberezkin/fast-deep-equal','https://github.com/tokio-rs/tokio','https://github.com/rich-harris/magic-string','https://github.com/webdriverio/webdriverio','https://github.com/axios/axios','https://github.com/numpy/numpy','https://github.com/vweevers/module-error','https://github.com/colorjs/color-name','https://github.com/brooooooklyn/snappy','https://github.com/import-js/eslint-plugin-import','https://github.com/vercel/swr','https://github.com/mysticatea/abort-controller','https://github.com/getsentry/sentry-javascript','https://github.com/humanwhocodes/object-schema','https://github.com/streetsidesoftware/cspell','https://github.com/boa-dev/boa','https://github.com/pandas-dev/pandas','https://github.com/inikulin/parse5','https://github.com/nodeca/js-yaml','https://github.com/formatjs/formatjs','https://github.com/yahoo/serialize-javascript','https://github.com/mmcloughlin/addchain','https://github.com/kaelzhang/node-ignore','https://github.com/tklauser/go-sysconf','https://github.com/vitest-dev/vitest','https://github.com/google/flatbuffers','https://github.com/biomejs/biome','https://github.com/fastify/fastify','https://github.com/servo/rust-smallvec','https://github.com/alexeyraspopov/picocolors','https://github.com/supranational/blst','https://github.com/status-im/nimbus-eth2','https://github.com/safe-global/safe-smart-account','https://github.com/eth-infinitism/account-abstraction','https://github.com/crate-crypto/go-ipa','https://github.com/paradigmxyz/reth','https://github.com/libp2p/go-libp2p','https://github.com/bluealloy/revm','https://github.com/ethereumjs/ethereumjs-monorepo','https://github.com/quic-go/quic-go','https://github.com/grandinetech/grandine','https://github.com/ethereum/solc-js','https://github.com/erigontech/erigon','https://github.com/alloy-rs/core','https://github.com/ajv-validator/ajv','https://github.com/ethereum/web3.py')

Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
# Querying data for valid dataset

# replace with your project name ↘↘↘
%%bigquery df_oso_valid --project hip-transducer-446111-g7

with watcher_count as (
  select
  project_id,
  artifact_id,
  artifact_url,
  watcher_count
from `oso_production.repositories_v0`
),

artifact_webpage as (
  select
    project_id,
    min(artifact_name) as webpage
  from `oso_production.artifacts_by_project_v1`
  where (artifact_source = 'WWW')
    and (artifact_name not like '%https://opencollective.com/%')
    and (artifact_name not like '%https://github.com/%')
    and (artifact_name not like '%http://discord.gg%')
  group by project_id
),

artifact_twitter as (
  select
    project_id,
    min(artifact_name) as twitter
  from `oso_production.artifacts_by_project_v1`
  where artifact_source = 'TWITTER'
  group by project_id
),

funding as (
  select
    count(distinct grant_pool_name) as count_all_grant_pools,
    to_project_id
  from `oso_production.oss_funding_v0`
  group by to_project_id
),

github_metrics as (
  select
    project_id,
    repository_count,
    developer_count,
    contributor_count
  from `oso_production.code_metrics_by_project_v1`
)

select
  watcher_count.artifact_url as url,
  watcher_count.watcher_count,
  artifact_webpage.webpage,
  artifact_twitter.twitter,
  funding.count_all_grant_pools,
  github_metrics.repository_count,
  github_metrics.developer_count,
  github_metrics.contributor_count
 from watcher_count
    left join artifact_webpage
      on watcher_count.project_id = artifact_webpage.project_id
    left join artifact_twitter
      on watcher_count.project_id = artifact_twitter.project_id
    left join funding
      on watcher_count.project_id = funding.to_project_id
    left join github_metrics
      on watcher_count.project_id = github_metrics.project_id

  -- Add the list of relevant repo_urls as a where clause
    where artifact_url in ('https://github.com/supranational/blst', 'https://github.com/prysmaticlabs/prysm', 'https://github.com/crate-crypto/go-ipa', 'https://github.com/ethereum/go-ethereum', 'https://github.com/safe-global/safe-smart-account', 'https://github.com/wevm/viem', 'https://github.com/nomicfoundation/hardhat', 'https://github.com/ethers-io/ethers.js', 'https://github.com/sheetjs/js-crc32', 'https://github.com/webdriverio/webdriverio', 'https://github.com/vercel/swr', 'https://github.com/vuejs/vue', 'https://github.com/walletconnect/walletconnect-monorepo', 'https://github.com/redux-saga/redux-saga', 'https://github.com/vweevers/module-error', 'https://github.com/wooorm/markdown-table', 'https://github.com/rollup/rollup', 'https://github.com/webpack/webpack', 'https://github.com/yahoo/serialize-javascript', 'https://github.com/qix-/color-convert', 'https://github.com/web3/web3.js', 'https://github.com/streetsidesoftware/cspell', 'https://github.com/pnpm/cmd-shim', 'https://github.com/salesforce/tough-cookie', 'https://github.com/sindresorhus/type-fest', 'https://github.com/rich-harris/magic-string', 'https://github.com/zloirock/core-js', 'https://github.com/prettier/prettier', 'https://github.com/pandas-dev/pandas', 'https://github.com/tokio-rs/tokio', 'https://github.com/pion/webrtc', 'https://github.com/tklauser/go-sysconf', 'https://github.com/webreflection/flatted', 'https://github.com/typescript-eslint/typescript-eslint', 'https://github.com/xtuc/webassemblyjs', 'https://github.com/ethereum/solidity', 'https://github.com/wighawag/hardhat-deploy', 'https://github.com/vyperlang/vyper', 'https://github.com/sigp/lighthouse', 'https://github.com/protofire/solhint', 'https://github.com/prettier-solidity/prettier-plugin-solidity', 'https://github.com/status-im/nimbus-eth2', 'https://github.com/paradigmxyz/reth', 'https://github.com/quic-go/quic-go', 'https://github.com/mattn/go-isatty', 'https://github.com/inikulin/parse5', 'https://github.com/mysticatea/abort-controller', 'https://github.com/postcss/postcss', 'https://github.com/immerjs/immer', 'https://github.com/mochajs/mocha', 'https://github.com/import-js/eslint-plugin-import', 'https://github.com/humanwhocodes/object-schema', 'https://github.com/kaelzhang/node-ignore', 'https://github.com/numpy/numpy', 'https://github.com/brooooooklyn/snappy', 'https://github.com/servo/rust-smallvec', 'https://github.com/google/flatbuffers', 'https://github.com/chzyer/readline', 'https://github.com/jedisct1/go-minisign', 'https://github.com/mmcloughlin/addchain', 'https://github.com/floating-ui/floating-ui', 'https://github.com/reactivex/rxjs', 'https://github.com/fastify/fastify', 'https://github.com/dcodeio/long.js', 'https://github.com/facebook/react', 'https://github.com/ljharb/qs', 'https://github.com/nodeca/js-yaml', 'https://github.com/heim-rs/darwin-libproc', 'https://github.com/cssnano/cssnano', 'https://github.com/vitest-dev/vitest', 'https://github.com/gregberge/svgr', 'https://github.com/gulpjs/gulp', 'https://github.com/level/levelup', 'https://github.com/mikemcl/bignumber.js', 'https://github.com/eslint/eslint', 'https://github.com/node-fetch/node-fetch', 'https://github.com/openzeppelin/openzeppelin-contracts', 'https://github.com/libp2p/go-libp2p', 'https://github.com/pytest-dev/pytest', 'https://github.com/go-task/slim-sprig', 'https://github.com/browserslist/browserslist', 'https://github.com/debug-js/debug', 'https://github.com/eth-infinitism/account-abstraction', 'https://github.com/ipfs/js-ipfs', 'https://github.com/fb55/entities', 'https://github.com/lukeed/polka', 'https://github.com/emotion-js/emotion', 'https://github.com/yarnpkg/yarn', 'https://github.com/swc-project/swc', 'https://github.com/formatjs/formatjs', 'https://github.com/mafintosh/pump', 'https://github.com/mozilla/source-map', 'https://github.com/grandinetech/grandine', 'https://github.com/motdotla/dotenv', 'https://github.com/epoberezkin/fast-deep-equal', 'https://github.com/ethereum/solc-js', 'https://github.com/electron/electron', 'https://github.com/erigontech/erigon', 'https://github.com/ethereum/remix-project', 'https://github.com/getsentry/sentry-javascript', 'https://github.com/biomejs/biome', 'https://github.com/coinbase/coinbase-wallet-sdk', 'https://github.com/colorjs/color-name', 'https://github.com/boa-dev/boa', 'https://github.com/marak/colors.js', 'https://github.com/clap-rs/clap', 'https://github.com/consensys/teku', 'https://github.com/bluealloy/revm', 'https://github.com/alloy-rs/core', 'https://github.com/bradfitz/iter', 'https://github.com/chainsafe/lodestar', 'https://github.com/ethereumjs/ethereumjs-monorepo', 'https://github.com/babel/babel', 'https://github.com/ajv-validator/ajv', 'https://github.com/alexeyraspopov/picocolors', 'https://github.com/axios/axios')

Query is running:   0%|          |

Downloading:   0%|          |

### Data Preparation (main_df, df_ratios, df_merged_ratios)



In [9]:
def merge_oso_df(df_oso, df):
    df_oso, df = df_oso.copy(), df.copy()
    df_oso['webpage'] = df_oso['webpage'].apply(lambda x: 0 if x == None else 1)
    df_oso['twitter'] = df_oso['twitter'].apply(lambda x: 0 if x == None else 1)
    df_oso[['developer_count', 'contributor_count']] = df_oso[
        ['developer_count', 'contributor_count']
    ].fillna(0)

    df_oso_a = df_oso.rename(columns={
        'url': 'project_a',
        'watcher_count': 'watcher_count_a',
        'description': 'description_a',
        'webpage': 'webpage_a',
        'twitter': 'twitter_a',
        'count_all_grant_pools': 'count_all_grant_pools_a',
        'repository_count': 'repository_count_a',
        'developer_count': 'developer_count_a',
        'contributor_count': 'contributor_count_a',
        'description': 'description_a'
        }
    )
    df_oso_b = df_oso.rename(columns={
        'url': 'project_b',
        'watcher_count': 'watcher_count_b',
        'description': 'description_b',
        'webpage': 'webpage_b',
        'twitter': 'twitter_b',
        'count_all_grant_pools': 'count_all_grant_pools_b',
        'repository_count': 'repository_count_b',
        'developer_count': 'developer_count_b',
        'contributor_count': 'contributor_count_b',
        'description': 'description_b'
        }
    )

    df = df.copy()

    df = df.drop(columns=[
        "total_funding_usd_a",
        "total_funding_usd_b",
        "total_funding_usd_since_2023_a",
        "total_funding_usd_since_2023_b",
        "list_of_dependents_in_oso_a",
        "list_of_dependents_in_oso_b",
        "list_of_packages_a",
        "list_of_packages_b"
        ],
    )

    df_merged = pd.merge(df, df_oso_a, on='project_a', how='left')
    df_merged = pd.merge(df_merged, df_oso_b, on='project_b', how='left')

    return df_merged

In [10]:
def get_final_datasets(df):
    df = df.copy()

    # ------------------------------------------------------------------------------
    # Make separate column per funder (True / False) for projects a

    list_of_unique_funders = ['Optimism', 'Gitcoin', 'Open Collective', 'Octant']

    df['Optimism'], df['Gitcoin'], df['Open Collective'], df['Octant'] = 0,0,0,0

    for index, row in df.iterrows():
      for val in row["list_of_funders_a"]:
        df.loc[index, val] = 1

    df = df.rename(
        columns={
            'Optimism': 'funder_Optimism_a',
            'Gitcoin': 'funder_Gitcoin_a',
            'Open Collective': 'funder_OpenCollective_a',
            'Octant': 'funder_Octant_a'
        }
    )

    # ------------------------------------------------------------------------------
    # Make separate column per funder (True / False) for projects b

    df['Optimism'], df['Gitcoin'], df['Open Collective'], df['Octant'] = 0,0,0,0

    for index, row in df.iterrows():
      for val in row["list_of_funders_b"]:
        df.loc[index, val] = 1

    df = df.rename(
        columns={
            'Optimism': 'funder_Optimism_b',
            'Gitcoin': 'funder_Gitcoin_b',
            'Open Collective': 'funder_OpenCollective_b',
            'Octant': 'funder_Octant_b'
        }
    )

    # ------------------------------------------------------------------------------
    # Change type to datetime and add year columns
    # Drop date columns

    df['year_created_a'] = pd.to_datetime(df['created_at_a']).dt.year
    df['year_updated_a'] = pd.to_datetime(df['updated_at_a']).dt.year

    df['year_created_b'] = pd.to_datetime(df['created_at_b']).dt.year
    df['year_updated_b'] = pd.to_datetime(df['updated_at_b']).dt.year

    drop_dt_cols = ['created_at_a', 'updated_at_a', 'created_at_b', 'updated_at_b']

    df = df.drop(columns=drop_dt_cols)

    # ------------------------------------------------------------------------------
    # Encode bool vals as 1 and 0

    df['is_fork_a'] = df['is_fork_a'].astype(int)
    df['is_fork_b'] = df['is_fork_b'].astype(int)

    # ------------------------------------------------------------------------------
    # Encode category columns into integers

    label_encode_ab('maintainer_a', 'maintainer_b', df)
    label_encode_ab('language_a', 'language_b', df)

    # ------------------------------------------------------------------------------
    # Replace list of funders with length of list
    df['list_of_funders_a'] = df['list_of_funders_a'].apply(len)
    df['list_of_funders_b'] = df['list_of_funders_b'].apply(len)
    df = df.rename(
        columns={
            'list_of_funders_a': 'num_funders_train_df_a',
            'list_of_funders_b': 'num_funders_train_df_b'
        }
    )

    # ------------------------------------------------------------------------------
    # Create a column that checks if the dependencies language is the same

    df['same_language'] = (df['language_a'] == df['language_b']).astype(int)

    # ------------------------------------------------------------------------------
    # Drop columns not needed for training
    df = df.drop(columns=['project_a', 'project_b'])

    df_merged_ratios = df.copy()
    # Calculate ratios for columns in the list
    cols_for_ratios = [
        'star_count_a', 'fork_count_a', 'num_packages_a',
        'num_dependents_in_oso_a', 'num_repos_in_same_language_a',
        'watcher_count_a', 'count_all_grant_pools_a', 'repository_count_a',
    ]

    # For cols with too many 0 values
    cols_for_subtraction = [
        'developer_count_a', 'contributor_count_a'
    ]

    for col in cols_for_ratios:
      df_merged_ratios[col[:-1]+'ratio_a'] = df_merged_ratios[col] / (
          df_merged_ratios[col] + df_merged_ratios[col[:-1]+'b']
      )

    for col in cols_for_subtraction:
      df_merged_ratios[col[:-1]+'diff_a'] = df_merged_ratios[col] - df_merged_ratios[col[:-1]+'b']

    # Replace null values with 0
    df_merged_ratios[[
        'num_dependents_in_oso_ratio_a',
        'num_packages_ratio_a'
        ]] = df_merged_ratios[[
            'num_dependents_in_oso_ratio_a',
            'num_packages_ratio_a'
            ]].fillna(0)

    # Drop columns that are not ratios
    df_ratios = df_merged_ratios.drop(columns=cols_for_ratios+cols_for_subtraction)

    return df, df_merged_ratios, df_ratios

In [11]:
df_merged_train, df_merged_valid = merge_oso_df(df_oso_train, df_train), merge_oso_df(df_oso_valid, df_valid)

In [12]:
df_train, df_merged_ratios_train, df_ratios_train = get_final_datasets(df_merged_train)

In [13]:
df_valid, df_merged_ratios_valid, df_ratios_valid = get_final_datasets(df_merged_valid)

### Hist Gradient Boosting

In [14]:
df_hgb = df_merged_ratios_train.copy()

categorical_features = [
    'maintainer_a', 'language_a', 'is_fork_a', 'webpage_a', 'twitter_a',
    'funder_Optimism_a', 'funder_Gitcoin_a', 'funder_OpenCollective_a',
    'funder_Octant_a', 'year_created_a', 'year_updated_a', 'same_language',
    'maintainer_b', 'language_b', 'is_fork_b', 'webpage_b', 'twitter_b',
    'funder_Optimism_b', 'funder_Gitcoin_b', 'funder_OpenCollective_b',
    'funder_Octant_b', 'year_created_b', 'year_updated_b'
]

X_train = df_hgb.drop(columns=['weight_a', 'weight_b'])
y_train = df_hgb[['id', 'weight_a']]
X_test = df_merged_ratios_valid.copy()

params = {
  "max_leaf_nodes": 15,
  "random_state": 17,
  "l2_regularization": 0.05,
  "categorical_features": categorical_features
}

hist_regressor = HistGradientBoostingRegressor(
  early_stopping=False,
  **params
)

hist_regressor.fit(X_train, y_train['weight_a'])

y_pred = hist_regressor.predict(X_test)

y_pred_ranged = fix_pred_range(y_pred)
final_y_pred = pd.DataFrame({
    'id': np.array(X_test['id']),
    'pred': np.array(y_pred_ranged).round(11)
})

In [16]:
final_y_pred.to_csv('pred.csv', index=False)